In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [2]:
# Load and combine data
df1 = pd.read_excel('data/singPres.xlsx')
df2 = pd.read_excel('data/subj_sessDays.xlsx')
df1['time'] = 0
for idx in range(len(df1)):
    subj    = df1.subj[idx]
    session = df1.session[idx]
    df1.iloc[idx,-1] = df2[np.logical_and(df2.subj == subj, df2.ses == session)].time.item()
df1

,Unnamed: 0,subj,session,pres,corr,time
0,0,soe101,1,1,0.050844,0
1,1,soe103,1,1,0.065902,0
2,2,soe104,1,1,0.041969,0
3,3,soe105,1,1,0.048064,0
4,4,soe106,1,1,0.024797,0
...,...,...,...,...,...,...
235,235,soe141,3,2,0.015822,25
236,236,soe142,3,2,0.019752,13
237,237,soe143,3,2,0.002006,157
238,238,soe144,3,2,0.018284,14


In [3]:
# Compute model
'''
This should re-generate the 2 way RM ANOVA
'''
model = smf.mixedlm('corr ~ session + pres + session*pres', df1, groups = df1['subj'])
mdf = model.fit(method=['lbfgs'])
mdf.summary()

/home/smazurchuk/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
=======================================================
Model:             MixedLM Dependent Variable: corr    
No. Observations:  240     Method:             REML    
No. Groups:        40      Scale:              0.0001  
Min. group size:   6       Log-Likelihood:     736.8223
Max. group size:   6       Converged:          Yes     
Mean group size:   6.0                                 
-------------------------------------------------------
             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.062    0.005 12.675 0.000  0.052  0.072
session      -0.011    0.002 -5.148 0.000 -0.015 -0.007
pres         -0.020    0.003 -6.725 0.000 -0.026 -0.014
session:pres  0.003    0.001  2.456 0.014  0.001  0.006
Group Var     0.000    0.003                           
=======================================================

"""

In [4]:
'''
This adds time as a random effect
'''
model = smf.mixedlm('corr ~ session + pres + session*pres', df1, groups = df1['subj'],re_formula="~ time")
mdf = model.fit(method=['lbfgs'])
mdf.summary()

/home/smazurchuk/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
===========================================================
Model:               MixedLM  Dependent Variable:  corr    
No. Observations:    240      Method:              REML    
No. Groups:          40       Scale:               0.0001  
Min. group size:     6        Log-Likelihood:      736.2771
Max. group size:     6        Converged:           Yes     
Mean group size:     6.0                                   
-----------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept         0.062    0.005 12.877 0.000  0.053  0.072
session          -0.011    0.002 -5.336 0.000 -0.015 -0.007
pres             -0.020    0.003 -6.959 0.000 -0.026 -0.014
session:pres      0.003    0.001  2.541 0.011  0.001  0.006
Group Var         0.000    0.005                           
Group x time Cov  0.000    0.000                           
time Var          0.000    0.000                           
===========================================================

"""

In [5]:
'''
This adds time as a fixed effect
'''
model = smf.mixedlm('corr ~ session + pres + time + session*pres', df1, groups = df1['subj'], re_formula="~ time")
mdf = model.fit(method=['lbfgs'])
mdf.summary()

/home/smazurchuk/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/smazurchuk/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
===========================================================
Model:               MixedLM  Dependent Variable:  corr    
No. Observations:    240      Method:              REML    
No. Groups:          40       Scale:               0.0001  
Min. group size:     6        Log-Likelihood:      726.8486
Max. group size:     6        Converged:           Yes     
Mean group size:     6.0                                   
-----------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept         0.063    0.005 12.876 0.000  0.053  0.072
session          -0.012    0.002 -5.420 0.000 -0.016 -0.007
pres             -0.020    0.003 -6.947 0.000 -0.026 -0.014
time              0.000    0.000  0.966 0.334 -0.000  0.000
session:pres      0.003    0.001  2.537 0.011  0.001  0.006
Group Var         0.000    0.006                           
Group x time Cov  0.000    0.000                           
time Var          0.000                                    
===========================================================

"""

In [6]:
'''
This adds time as a fixed effect with 0 intercept
'''
model = smf.mixedlm('corr ~ session + pres + time + session*pres', df1, groups = df1['subj'],re_formula="~ 0 + time")
mdf = model.fit(method=['lbfgs'])
mdf.summary()

/home/smazurchuk/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/home/smazurchuk/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
=======================================================
Model:             MixedLM Dependent Variable: corr    
No. Observations:  240     Method:             REML    
No. Groups:        40      Scale:              0.0001  
Min. group size:   6       Log-Likelihood:     690.5342
Max. group size:   6       Converged:          Yes     
Mean group size:   6.0                                 
-------------------------------------------------------
             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.064    0.006 10.583 0.000  0.052  0.076
session      -0.013    0.003 -4.308 0.000 -0.018 -0.007
pres         -0.020    0.004 -5.315 0.000 -0.027 -0.013
time          0.000    0.000  1.101 0.271 -0.000  0.000
session:pres  0.003    0.002  1.941 0.052 -0.000  0.007
time Var      0.000    0.000                           
=======================================================

"""

In [ ]:
tmp